<a href="https://colab.research.google.com/github/PedroH-07/Checkpoint1-SERS/blob/main/CP01_SERS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd

In [18]:
df = pd.read_csv("/content/household_power_consumption.txt", sep=';')

FileNotFoundError: [Errno 2] No such file or directory: '/content/household_power_consumption.txt'

# **01 - Carregue o dataset e exibe as 10 primeiras linhas**

In [ ]:
df.head(11)

## **02 diferença entre Global_active_power e Global_reactive_power**

Global_active_power: Esta coluna representa a potência ativa total consumida pela residência em quilowatts (kW). A potência ativa é a potência real que realiza trabalho útil, como acender lâmpadas, fazer funcionar motores, etc. É a energia que você realmente "usa" e pela qual paga na sua conta de eletricidade.

Global_reactive_power: Esta coluna representa a potência reativa total consumida pela residência em quilovolt-ampère reativo (kVAr). A potência reativa é a potência que não realiza trabalho útil diretamente, mas é necessária para criar campos magnéticos em equipamentos como motores, transformadores e reatores. Ela não é convertida em trabalho, mas é necessária para que a potência ativa possa ser entregue.

## **03 Verifique se tem valores ausentes e quantifique-os**

In [ ]:
missing_values = df.isnull().sum()
display(missing_values)

# **04 Converta a coluna Date para o tipo datetime e cria uma nova coluna com o dia da semana correspondente**

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['DayOfWeek'] = df['Date'].dt.day_name()
display(df.head())

# **05 Filtre os registros apenas do ano 2007 e calcule a média de consumo diário de Gblobal_active_power**

In [ ]:
df['Global_active_power'] = pd.to_numeric(df['Global_active_power'], errors='coerce')
df_2007 = df[df['Date'].dt.year == 2007]

daily_avg_power_2007 = df_2007.groupby('Date')['Global_active_power'].mean()
display(daily_avg_power_2007)

# **06. Gere um gráfico de linha mostrando a variação de Global_active_power em um único dia à sua escolha.**



In [ ]:
import matplotlib.pyplot as plt

date_to_plot = df['Date'].iloc[0]
df_single_day = df[df['Date'] == date_to_plot].copy()

df_single_day['Datetime'] = pd.to_datetime(df_single_day['Date'].astype(str) + ' ' + df_single_day['Time'])

plt.figure(figsize=(12, 6))
plt.plot(df_single_day['Datetime'], df_single_day['Global_active_power'])
plt.xlabel('Time')
plt.ylabel('Global Active Power (kW)')
plt.title(f'Global Active Power Variation on {date_to_plot.strftime("%Y-%m-%d")}')
plt.grid(True)
plt.show()

# **07. Crie um histograma da variável Voltage. O que pode ser observado sobre sua distribuição?**

In [ ]:
import matplotlib.pyplot as plt

df['Voltage'] = pd.to_numeric(df['Voltage'], errors='coerce')

df.dropna(subset=['Voltage'], inplace=True)

plt.figure(figsize=(10, 6))
plt.hist(df['Voltage'], bins=50, edgecolor='black')
plt.xlabel('Voltage (V)')
plt.ylabel('Frequency')
plt.title('Distribution of Voltage')
plt.grid(True)
plt.show()

# **08. Calcule o consumo médio por mês em todo o período disponível no dataset.**

In [ ]:
df['Month'] = df['Date'].dt.to_period('M')
monthly_avg_power = df.groupby('Month')['Global_active_power'].mean()
display(monthly_avg_power)

# **09. Identifique o dia com maior consumo de energia ativa global (Global_active_power).**

In [ ]:
daily_power_sum = df.groupby('Date')['Global_active_power'].sum()
day_with_highest_consumption = daily_power_sum.idxmax()
highest_consumption_value = daily_power_sum.max()

print(f"O dia com maior consumo de energia ativa global é: {day_with_highest_consumption.strftime('%Y-%m-%d')}")
print(f"Valor do maior consumo: {highest_consumption_value:.2f} kW")

# **10. Compare o consumo médio de energia ativa global em dias de semana versus finais de semana.**

In [ ]:
df['DayType'] = df['DayOfWeek'].apply(lambda x: 'Final de semana' if x in ['Saturday', 'Sunday'] else 'Dia de semana')

average_power_by_daytype = df.groupby('DayType')['Global_active_power'].mean()

display(average_power_by_daytype)

# ***15. Compare o consumo entre os meses de verão e inverno (no hemisfério norte).***

In [ ]:
summer_months = [6, 7, 8]
df_summer = df[df['Date'].dt.month.isin(summer_months)]

average_power_summer = df_summer['Global_active_power'].mean()

winter_months = [12, 1, 2]
df_winter = df[df['Date'].dt.month.isin(winter_months)]

average_power_winter = df_winter['Global_active_power'].mean()

print(f"Consumo médio de energia ativa global no verão (Hemisfério Norte): {average_power_summer:.2f} kW")
print(f"Consumo médio de energia ativa global no inverno (Hemisfério Norte): {average_power_winter:.2f} kW")

# ***16. Aplique uma amostragem aleatória de 1% dos dados e verifique se a distribuição de Global_active_power é semelhante à da base completa***

In [ ]:
df_sample = df.sample(frac=0.01, random_state=42)

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.hist(df['Global_active_power'], bins=50, edgecolor='black')
plt.xlabel('Global Active Power (kW)')
plt.ylabel('Frequência')
plt.title('Distribuição da Potência Ativa Global (Full Dataset)')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.hist(df_sample['Global_active_power'], bins=50, edgecolor='black')
plt.xlabel('Global Active Power (kW)')
plt.ylabel('Frequência')
plt.title('Distribuição da Potência Ativa Global (Amostra de 1%)')
plt.grid(True)

plt.tight_layout()

# ***17. Utilize uma técnica de normalização (Min-Max Scaling) para padronizar as variáveis numéricas principais.***



In [ ]:
from sklearn.preprocessing import MinMaxScaler

numerical_cols = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']

scaler = MinMaxScaler()

df_scaled = df.copy()
df_scaled[numerical_cols] = scaler.fit_transform(df_scaled[numerical_cols])

display(df_scaled.head())

# ***18. Aplique K-means para segmentar os dias em 3 grupos distintos de consumo elétrico. Interprete os resultados.***

In [ ]:
daily_consumption = df.groupby('Date')['Global_active_power'].sum().reset_index()
display(daily_consumption.head())

In [ ]:
daily_power = daily_consumption['Global_active_power']

X = daily_power.values.reshape(-1, 1)

print(f"Formato dos dados para K-means: {X.shape}")

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)

kmeans.fit(X)

In [ ]:
daily_consumption['Cluster'] = kmeans.labels_
display(daily_consumption.head())

In [ ]:
average_consumption_per_cluster = daily_consumption.groupby('Cluster')['Global_active_power'].mean()

print("Potência Ativa Global Média por Cluster:")
display(average_consumption_per_cluster)

plt.figure(figsize=(10, 6))
plt.scatter(daily_consumption.index, daily_consumption['Global_active_power'], c=daily_consumption['Cluster'], cmap='viridis')
plt.xlabel('Índice de Dias')
plt.ylabel('Global Active Power (kW)')
plt.title('Agrupamento K-means do Consumo Diário de Potência Ativa Global')
plt.colorbar(label='Cluster')
plt.grid(True)
plt.show()

# ***19. Realize uma decomposição de série temporal (tendência, sazonalidade e resíduo) para Global_active_power em um período de 6 meses.*`**

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

start_date = '2007-01-01'
end_date = '2007-06-30'

df_period = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].copy()

display(df_period.head())

In [ ]:
df_period['Datetime'] = pd.to_datetime(df_period['Date'].astype(str) + ' ' + df_period['Time'])
df_period.set_index('Datetime', inplace=True)

daily_power_series = df_period['Global_active_power'].resample('D').mean()

display(daily_power_series.head())

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

daily_power_series_filled = daily_power_series.interpolate()

decomposition = seasonal_decompose(daily_power_series_filled, model='additive', period=7)

In [ ]:
fig = decomposition.plot()
fig.set_size_inches(12, 8)
plt.suptitle('Decomposição da Série Temporal da Potência Ativa Global Diária (6 Meses)', y=1.02)
plt.show()

# ***20. Treine um modelo de regressão linear simples para prever Global_active_power a partir de Global_intensity. Avalie o erro do modelo***

In [ ]:
X = df[['Global_intensity']]
y = df['Global_active_power']

display(X.head())
display(y.head())

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

print("Primeiras 5 previsões:")
print(y_pred[:5])

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print(f"Erro Quadrático Médio (MSE): {mse:.4f}")
print(f"R-quadrado (R2): {r2:.4f}")

# ***21***

In [ ]:
df['Datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'], format='%Y-%m-%d %H:%M:%S')
df.set_index('Datetime', inplace=True)
df.drop(['Date', 'Time'], axis=1, inplace=True)
display(df.head())

In [ ]:
df_hourly = df['Global_active_power'].resample('H').mean()
display(df_hourly.head())

In [ ]:
df_hourly = df['Global_active_power'].resample('H').mean()
hourly_avg_power = df_hourly.groupby(df_hourly.index.hour).mean()
peak_hours = hourly_avg_power.sort_values(ascending=False)
display(peak_hours)

# ***22***

In [ ]:
from statsmodels.tsa.stattools import acf

autocorr_lag1 = acf(df_hourly, nlags=1)[1]

autocorr_lag24 = acf(df_hourly, nlags=24)[24]

autocorr_lag48 = acf(df_hourly, nlags=48)[48]

print(f"Autocorrelação no atraso de 1 horas: {autocorr_lag1:.4f}")
print(f"Autocorrelação no atraso de 24 horas: {autocorr_lag24:.4f}")
print(f"Autocorrelação no atraso de 48 horas: {autocorr_lag48:.4f}")

In [ ]:
from statsmodels.tsa.stattools import acf

autocorr_lag1 = acf(df_hourly, nlags=1)[1]

autocorr_lag24 = acf(df_hourly, nlags=24)[24]

autocorr_lag48 = acf(df_hourly, nlags=48)[48]

print(f"Autocorrelação no atraso de 1 horas: {autocorr_lag1:.4f}")
print(f"Autocorrelação no atraso de 24 horas: {autocorr_lag24:.4f}")
print(f"Autocorrelação no atraso de 48 horas: {autocorr_lag48:.4f}")

In [ ]:
df_hourly_filled = df_hourly.interpolate(method='linear')

autocorr_lag1 = acf(df_hourly_filled, nlags=1)[1]

autocorr_lag24 = acf(df_hourly_filled, nlags=24)[24]

autocorr_lag48 = acf(df_hourly_filled, nlags=48)[48]

print(f"Autocorrelação no atraso de 1 horas: {autocorr_lag1:.4f}")
print(f"Autocorrelação no atraso de 24 horas: {autocorr_lag24:.4f}")
print(f"Autocorrelação no atraso de 48 horas: {autocorr_lag48:.4f}")

In [ ]:
print("Interpretação dos Valores de Autocorrelação:")
print(f"- Autocorrelação no atraso de 1 horas ({autocorr_lag1:.4f}): Esse valor é relativamente alto e positivo. Ele indica uma forte correlação positiva entre o consumo de potência ativa global em uma determinada hora e o consumo na hora anterior. Isso sugere que o consumo de energia tende a ser semelhante de uma hora para a outra.")
print(f"- Autocorrelação no atraso de 24 horas ({autocorr_lag24:.4f}): Esse valor também é positivo e moderadamente alto, embora menor do que no atraso de 1 hora. Uma autocorrelação positiva no atraso de 24 horas sugere uma correlação positiva entre o consumo de energia em uma determinada hora e o consumo exatamente 24 horas antes (a mesma hora no dia anterior). Isso é um forte indicador de um padrão diário repetitivo no consumo de energia.")
print(f"- Autocorrelação no atraso de 48 horas ({autocorr_lag48:.4f}): Esse valor é positivo, mas um pouco menor do que no atraso de 24 horas. Ele indica uma correlação positiva entre o consumo de energia em uma determinada hora e o consumo 48 horas antes (a mesma hora dois dias antes). O fato de ainda ser positivo e relativamente alto reforça a presença de padrões recorrentes, embora potencialmente com uma correlação um pouco menos forte do que o atraso diário imediato.")

print("\nConclusão:")
print("Com base nos valores de autocorrelação, particularmente o valor positivo significativo no atraso de 24 horas, existem padrões diários repetitivos evidentes na série temporal de Potência Ativa Global. O consumo de energia tende a seguir um padrão semelhante a cada dia, com o consumo em uma hora específica estando positivamente correlacionado com o consumo na mesma hora do dia anterior.")

# ***Pergunta: existem padrões repetidos diariamente?***


Com base nos valores de autocorrelação, particularmente o valor positivo significativo no lag de 24 horas, existem padrões diários repetidos notáveis na série temporal de Global_active_power. O consumo de energia tende a seguir um padrão semelhante a cada dia, com o consumo em uma hora específica sendo positivamente correlacionado com o consumo na mesma hora do dia anterior.

# ***23***

In [ ]:
features = df[['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']]
display(features.head())

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=42)

df_pca = pd.DataFrame(pca.fit_transform(features), columns=['PC1', 'PC2'])
display(df_pca.head())

In [ ]:
explained_variance_ratio = pca.explained_variance_ratio_
print("Proporção da variância explicada de cada componente:")
print(explained_variance_ratio)

cumulative_explained_variance = explained_variance_ratio.sum()
print(f"\nProporção acumulada da variância explicada pelos dois primeiros componentes: {cumulative_explained_variance:.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.scatter(df_pca['PC1'], df_pca['PC2'], alpha=0.5, s=1)
plt.xlabel('Principal Componente 1')
plt.ylabel('Principal Componente 2')
plt.title('PCA dos Dados de Consumo de Energia (2 Componentes)')
plt.grid(True)
plt.show()

# ***24***

In [ ]:
daily_consumption_indexed = daily_consumption.set_index('Data')

df['Cluster'] = df.index.normalize().map(daily_consumption_indexed['Cluster'])

df_pca = df_pca.set_index(df.index)
df_pca['Cluster'] = df['Cluster']

df_pca.dropna(subset=['Cluster'], inplace=True)

df_pca['Cluster'] = df_pca['Cluster'].astype(int)

display(df_pca.head())

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(df_pca['PC1'], df_pca['PC2'], c=df_pca['Cluster'], cmap='viridis', s=1, alpha=0.5)
plt.xlabel('Principal Componente 1')
plt.ylabel('Principal Componente 2')
plt.title('Clusters de K-means no Espaço PCA')
plt.colorbar(label='Cluster')
plt.grid(True)
plt.show()

# ***Os grupos se separam?***

Observando o gráfico de dispersão dos clusters de K-means no espaço ACP, os grupos não parecem estar claramente separados. Há uma sobreposição significativa entre os clusters, sugerindo que as características usadas para o agrupamento, quando reduzidas a dois componentes principais, não diferenciam os grupos de forma distinta.

# ***25***

In [ ]:
from sklearn.model_selection import train_test_split

features_voltage = df[['Voltage']]
target_power = df['Global_active_power']

X_train_voltage, X_test_voltage, y_train_power, y_test_power = train_test_split(
    features_voltage, target_power, test_size=0.2, random_state=42
)

print(f"Formato de X_train_voltage: {X_train_voltage.shape}")
print(f"Formato de X_test_voltage: {X_test_voltage.shape}")
print(f"Formato de y_train_power: {y_train_power.shape}")
print(f"Formato de y_test_power: {y_test_power.shape}")

In [ ]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train_voltage, y_train_power)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly_features = PolynomialFeatures(degree=2)

X_train_voltage_poly = poly_features.fit_transform(X_train_voltage)
X_test_voltage_poly = poly_features.transform(X_test_voltage)

poly_model = LinearRegression()
poly_model.fit(X_train_voltage_poly, y_train_power)

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

y_pred_linear = linear_model.predict(X_test_voltage)

rmse_linear = np.sqrt(mean_squared_error(y_test_power, y_pred_linear))

y_pred_poly = poly_model.predict(X_test_voltage_poly)

rmse_poly = np.sqrt(mean_squared_error(y_test_power, y_pred_poly))

print(f"Raiz do Erro Quadrático Médio (RMSE) para o Modelo Linear: {rmse_linear:.4f}")
print(f"Raiz do Erro Quadrático Médio (RMSE) para o Modelo Polinomial: {rmse_poly:.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.scatter(X_test_voltage, y_test_power, alpha=0.5, s=1, label='Original Data')

X_test_voltage_sorted = X_test_voltage.sort_values(by='Voltage')

y_pred_linear_sorted = linear_model.predict(X_test_voltage_sorted)

X_test_voltage_sorted_poly = poly_features.transform(X_test_voltage_sorted)
y_pred_poly_sorted = poly_model.predict(X_test_voltage_sorted_poly)

plt.plot(X_test_voltage_sorted, y_pred_linear_sorted, color='red', label='Linear Fit')

plt.plot(X_test_voltage_sorted, y_pred_poly_sorted, color='green', label='Polynomial Fit')

plt.xlabel('Tensão (V)')
plt.ylabel('Potência Ativa Global (kW)')
plt.title('Ajuste de Regressão Linear vs Polinomial em Tensão e Potência Ativa Global')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print(f"Comparação do Desempenho dos Modelos (RMSE):")
print(f"RMSE do Modelo Linear: {rmse_linear:.4f}")
print(f"RMSE do Modelo Polinomial: {rmse_poly:.4f}")

if rmse_poly < rmse_linear:
    print("\nO modelo de Regressão Polinomial apresenta um RMSE menor e fornece um ajuste ligeiramente melhor com base nessa métrica.")
elif rmse_linear < rmse_poly:
    print("\nO modelo de Regressão Linear apresenta um RMSE menor e fornece um ajuste ligeiramente melhor com base nessa métrica.")
else:
    print("\nAmbos os modelos possuem o mesmo RMSE.")

print("\nInterpretação Visual das Curvas Ajustadas:")
print("Observando o gráfico, o modelo de regressão linear ajusta uma linha reta aos dados. Essa linha captura a tendência geral, mas não acompanha as nuances da distribuição dos dados.")
print("O modelo de regressão polinomial (grau 2) ajusta uma curva aos dados. Essa curva parece seguir a distribuição dos pontos de dados mais de perto do que a linha reta, especialmente em áreas onde os dados apresentam uma leve curvatura.")
print("Com base na inspeção visual, o modelo polinomial parece capturar melhor a relação entre Tensão e Potência Ativa Global do que o modelo linear simples.")

print("\nConclusão Geral:")
print("Tanto as avaliações quantitativas (RMSE) quanto qualitativas (visuais) sugerem que o modelo de regressão polinomial fornece um ajuste ligeiramente melhor aos dados para prever a Potência Ativa Global com base na Tensão, em comparação com o modelo linear simples. O valor menor de RMSE e a curva com melhor ajuste visual indicam que a incorporação de um termo polinomial para a Tensão melhora a capacidade do modelo de capturar o padrão subjacente.")